In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 3.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633978 sha256=8eac5b923723e17b1a9d90ecb6e1c71c96d7a30c7e200f336cb7a4982d5ecaa7
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate, GridSearchCV, KFold
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from surprise import SVDpp, accuracy
from surprise import SVD, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering

In [ ]:
import pickle


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# df = pd.read_csv('/content/drive/Othercomputers/My Laptop/OneDrive/CMPE 256/Project/train_triplets_large.txt', sep='\t', index_col=None, names = ['User','Song', 'Count'])
df = pd.read_csv('/content/drive/MyDrive/256 Project/train_triplets.txt', sep='\t', index_col=None, names = ['User','Song', 'Count'])
df.head()

,User,Song,Count
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAPDEY12A81C210A9,1
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFNSP12AF72A0E22,1
4,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBFOVM12A58A7D494,1


In [ ]:
# from plotly.offline import init_notebook_mode, plot, iplot
# import plotly.graph_objs as go
# init_notebook_mode(connected=True)

# data = df['Count'].value_counts().sort_index(ascending=False)
# trace = go.Bar(x = data.index,
#                text = ['{:.1f} %'.format(val) for val in (data.values / df.shape[0] * 100)],
#                textposition = 'auto',
#                textfont = dict(color = '#000000'),
#                y = data.values,
#                )
# # Create layout
# layout = dict(title = 'Distribution Of {} book-ratings'.format(df.shape[0]),
#               xaxis = dict(title = 'Counts'),
#               yaxis = dict(title = 'Occurences'))
# # Create plot
# fig = go.Figure(data=[trace], layout=layout)
# iplot(fig)

In [ ]:
df['Count'] = df["Count"].astype(np.int64)

In [ ]:
a = df["Count"].value_counts().sort_index()

In [ ]:
index_list = a.index.values.tolist()

In [ ]:
listindex = [i for i in index_list if i >= 1000 or i<=2]

In [ ]:
df = df[~df['Count'].isin(listindex)]

In [ ]:
min_count = min(df["Count"])
max_count = max(df["Count"])

In [ ]:
max_count

995

In [ ]:
# users = pd.read_csv('/content/drive/MyDrive/Music Recommendation/train_users.csv', sep=',', index_col=None, names = ['User','Count'])
users = pd.read_csv('/content/drive/MyDrive/256 Project/Music Recommendation/train_users.csv', sep=',', index_col=None, names = ['User','Count'], skiprows=1)
users

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,User,Count
0,User,Count
1,10c3da065896b3f9c10845aa62a788a56590fd84,166
2,6e8ac84fa420ef37839559491f664644bc7349b8,103
3,cd3b3a18ccc912a9d87dec984fe8a46fa6dd05ab,13
4,8b6d7a53a4311f9ecf43c09a99e261f378035b87,61
...,...,...
815450,16ae0e0818b4c3b99134380464b858b6b14b5ada,61
815451,1fa88d828dc104feb4e6bb11f360e6601178b2a4,34
815452,6b53f11ebc58dc591dc6c88fbee3c220386d1b80,16
815453,297d20c3f24a11c7264ae6249fb0bf5c303c5120,60


In [ ]:
users.drop(index=users.index[0], 
        axis=0, 
        inplace=True)

In [ ]:
users

,User,Count
1,10c3da065896b3f9c10845aa62a788a56590fd84,166
2,6e8ac84fa420ef37839559491f664644bc7349b8,103
3,cd3b3a18ccc912a9d87dec984fe8a46fa6dd05ab,13
4,8b6d7a53a4311f9ecf43c09a99e261f378035b87,61
5,ecd0416d0e4f292dc0570f5e6b87521e11622d68,84
...,...,...
815450,16ae0e0818b4c3b99134380464b858b6b14b5ada,61
815451,1fa88d828dc104feb4e6bb11f360e6601178b2a4,34
815452,6b53f11ebc58dc591dc6c88fbee3c220386d1b80,16
815453,297d20c3f24a11c7264ae6249fb0bf5c303c5120,60


In [ ]:
users_list = users.User.values.tolist()
# users_list

In [ ]:
new_df = df[df['User'].isin(users_list)]
new_df

,User,Song,Count
13,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SODDNQT12A6D4F5F7E,5
33,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOIZAZL12A6701C53B,5
37,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOKRIMP12A6D4F5DA3,5
45,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMGIYR12AB0187973,6
48,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOMZWUW12A8C1400BC,6
...,...,...,...
48373544,cf8289419383259189afe6bb50c5115fd84f1064,SOYSPLQ12AB0185D3F,7
48373545,cf8289419383259189afe6bb50c5115fd84f1064,SOZCDWG12A6D4F81E1,6
48373564,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOOFYTN12A6D4F9B35,4
48373574,b7815dbb206eb2831ce0fe040d0aa537e2e800f7,SOSQJWM12A6D4F79E0,6


In [ ]:
new_df.to_csv('/content/drive/MyDrive/256 Project/Music Recommendation/select_users.csv')

In [ ]:
reader = Reader(rating_scale=(0, 9))
data = Dataset.load_from_df(new_df[['User', 'Song', 'Count']], reader)

In [ ]:
# sim_options = { 'name': 'cosine' ,'item_based':  True}
# knnbasic = KNNBasic(sim_options=sim_options)
# results = cross_validate(knnbasic, data, measures=['RMSE'], cv=3, verbose=False)

In [ ]:
svd = SVD(verbose=True, n_epochs=10)
# cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9


In [ ]:
with open('/content/drive/MyDrive/256 Project/Music Recommendation/svd_model.pkl', 'wb') as f:
  pickle.dump(svd, f)

In [ ]:
test_users = pd.read_csv('/content/drive/MyDrive/256 Project/Music Recommendation/test_users.csv', sep=',', index_col=None, names = ['User','Count'], skiprows=1)
test_users

In [ ]:
test_users_list = test_users.User.values.tolist()

In [ ]:
test_df = df[df['User'].isin(test_users_list)]

In [ ]:
recs = test[['User','Song']]

In [ ]:
values = []
for index, row in recs.iterrows():
  pred = svd.predict(uid=row['User'], iid=row['Song'])
  values.append(round(pred.est, 2))

In [ ]:
recs['Count'] = values

In [ ]:
recs.head()

In [ ]:
accuracy.rmse(predictions)

In [ ]:
# benchmark = []
# # Iterate over all algorithms
# for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
#     # Perform cross validation
#     results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
#     # Get results & append algorithm name
#     tmp = pd.DataFrame.from_dict(results).mean(axis=0)
#     tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
#     benchmark.append(tmp)
    
# pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')  

In [ ]:
plt.plot(history.history["loss"])
plt.plot(history.history["val_loss"])
plt.title("model loss")
plt.ylabel("loss")
plt.xlabel("epoch")
plt.legend(["train", "test"], loc="upper left")
plt.show()